In [21]:
from huggingface_hub import hf_hub_download
import shutil
from pathlib import Path

DATASET_DIR = Path.cwd() / "dataset"
JSON_ANNOTATED_DIR = DATASET_DIR / "ResumesJsonAnnotated"
PDFS_DIR = DATASET_DIR / "ResumesPDF"

if DATASET_DIR.exists():
    print("Dataset already loaded")
else:
    DATASET_DIR.mkdir()
    annotated_zip = hf_hub_download(
        repo_id="Mehyaar/Annotated_NER_PDF_Resumes",
        repo_type="dataset",
        filename="ResumesJsonAnnotated.zip",
    )
    shutil.unpack_archive(annotated_zip, DATASET_DIR)
    print("Loaded json annotated resumes")
    pdfs_zip = hf_hub_download(
        repo_id="Mehyaar/Annotated_NER_PDF_Resumes",
        repo_type="dataset",
        filename="ResumesPDF.zip",
    )
    shutil.unpack_archive(pdfs_zip, DATASET_DIR)
    print("Loaded source pdfs")

ResumesJsonAnnotated.zip:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

Loaded json annotated resumes


ResumesPDF.zip:   0%|          | 0.00/586M [00:00<?, ?B/s]

Loaded source pdfs


In [22]:
import json
import os

data = []
for filename in os.listdir(JSON_ANNOTATED_DIR):
    if filename.endswith(".json"):
        with open(os.path.join(JSON_ANNOTATED_DIR, filename), "r") as file:
            data.append(json.load(file))

first_cv = data[0]
text = first_cv["text"]
annotations = first_cv["annotations"]

print(f"Text: {text}")
print(f"Annotations: {annotations}")

Text:   Roy A. Jaleco Jr.                                E-mail: ra.jaleco@yahoo.com § Phone:  +97158 882 6544                   440 The Centre Residence Al Muteena Deira Dubai, UAE EDUCATION Bachelor of Science in Information Technology                                                2006 — 2013 Adamson University Secondary                                                                                                              2001 — 2006 Grant’s Apostolic Institute Primary                                                                                                                 1996 — 2002 Grant’s Apostolic Institute  WORK EXPERIENCE Destinations of the World                                                       January 2017- February 2018 Data Entry Operator Mandaluyong, Philippines  • Accommodating requests from Partner Hotels all over the world together with the company’s Contract managers. • Process Updates based on Hotel’s preference. • Creation of Promotional Offers for